In [38]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from gensim.models import HdpModel
from gensim.corpora import Dictionary
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [60]:
# # Sample data
# data = {
#     'Ulasan': [
#         "Aplikasi yg sangat tdk rekomendasi,tdk sesuai dengan iklan. Penagihnya sangat meresahkan dan tidak ramah sama sekali kepada pelanggan.",
#         "Pengiriman barang sangat cepat, namun produk yang diterima tidak sesuai dengan deskripsi.",
#         "Sangat puas dengan pelayanan pelanggan, respon cepat dan membantu.",
#         "Kualitas produknya bagus, sesuai dengan harga yang ditawarkan.",
#         "Harga terlalu mahal untuk kualitas produk yang diberikan.",
#         "Layanan pelanggan sangat buruk, sulit dihubungi dan tidak membantu.",
#         "Sangat kecewa dengan pelayanan mereka, tidak sesuai dengan yang dijanjikan.",
#         "Pengiriman sangat lambat, produk juga tidak sesuai dengan yang diharapkan.",
#         "Saya sangat senang dengan produk ini, sudah membelinya beberapa kali.",
#         "Tidak akan merekomendasikan produk ini kepada teman, kualitasnya sangat buruk."
#     ],
#     'Sentimen': ['Negatif', 'Negatif', 'Positif', 'Positif', 'Negatif',
#                  'Negatif', 'Negatif', 'Negatif', 'Positif', 'Negatif']
# }

# # Create DataFrame
# df = pd.DataFrame(data)
# df

,Ulasan,Sentimen
0,"Aplikasi yg sangat tdk rekomendasi,tdk sesuai ...",Negatif
1,"Pengiriman barang sangat cepat, namun produk y...",Negatif
2,"Sangat puas dengan pelayanan pelanggan, respon...",Positif
3,"Kualitas produknya bagus, sesuai dengan harga ...",Positif
4,Harga terlalu mahal untuk kualitas produk yang...,Negatif
5,"Layanan pelanggan sangat buruk, sulit dihubung...",Negatif
6,"Sangat kecewa dengan pelayanan mereka, tidak s...",Negatif
7,"Pengiriman sangat lambat, produk juga tidak se...",Negatif
8,"Saya sangat senang dengan produk ini, sudah me...",Positif
9,Tidak akan merekomendasikan produk ini kepada ...,Negatif


In [72]:
df = pd.read_csv('../data/dataHasilPreprocessing/hasilPreprocessing1.csv')
df

,Ulasan,Sentimen
0,ulas,Negatif
1,NaN,Negatif
2,bahan aja warna navy nya beda,Negatif
3,kasih catat order warna kuning navy baca kirim...,Negatif
4,kecil,Negatif
...,...,...
990,barang selamat terima kasih bahan celana nya s...,Negatif
991,kecil,Negatif
992,pesan cuna barang rusak coba konfirmasi tanggap,Negatif
993,karet pinggang nya kencang,Negatif


In [73]:
df = df.drop_duplicates(subset=['Ulasan'])

In [74]:
df = df.dropna()

In [75]:
# Text vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Ulasan'])

In [76]:
# Hierarchical Dirichlet Process (HDP) for topic extraction
dictionary = Dictionary(df['Ulasan'].apply(str.split))
corpus = [dictionary.doc2bow(doc) for doc in df['Ulasan'].apply(str.split)]
hdp_model = HdpModel(corpus, dictionary)

# Get topic distribution for each document
topic_distributions = [dict(hdp_model[c]) for c in corpus]

# Convert topic distributions to dataframe
topics_df = pd.DataFrame(topic_distributions)

# Rename columns for better interpretation
topics_df = topics_df.rename(columns=lambda x: 'Topik{}'.format(x+1))

In [77]:
# Reorder columns based on column names
sorted_columns = sorted(topics_df.columns, key=lambda x: int(x[5:]))
topics_df = topics_df[sorted_columns]

# Combine with original dataframe
df_combined = pd.concat([df, topics_df], axis=1)

# Print combined dataframe
print("Combined DataFrame:")
df_combined

Combined DataFrame:


,Ulasan,Sentimen,Topik1,Topik2,Topik3,Topik4,Topik5,Topik6,Topik7,Topik8,...,Topik140,Topik141,Topik142,Topik143,Topik144,Topik145,Topik146,Topik147,Topik148,Topik150
0,ulas,Negatif,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bahan aja warna navy nya beda,Negatif,NaN,NaN,NaN,NaN,0.198594,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,kasih catat order warna kuning navy baca kirim...,Negatif,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,kecil,Negatif,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,tau layan,Negatif,NaN,NaN,0.670739,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# Training data using SVM
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X, df['Sentimen'])

# Evaluation - Classification Report
predictions = svm_classifier.predict(X)
print("\nClassification Report:")
print(classification_report(df['Sentimen'], predictions))


Classification Report:
              precision    recall  f1-score   support

     Negatif       0.99      0.61      0.76       135
     Positif       0.94      1.00      0.97       812

    accuracy                           0.94       947
   macro avg       0.96      0.81      0.86       947
weighted avg       0.95      0.94      0.94       947

